In [6]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category = DeprecationWarning)

import numpy as np
import pandas as pd

import os

from scipy.special import boxcox1p, inv_boxcox1p
from sklearn.preprocessing import RobustScaler

import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV

In [7]:
# 전체 경로
data_dir = os.getenv('HOME') + '/aiffel/kaggle_kakr_housing/data'

# 파일 경로
train_data_path = os.path.join(data_dir, 'train.csv')
test_data_path = os.path.join(data_dir, 'test.csv')

# 파일 변수에 저장
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

# 아이디 제거
del train_data['id']
del test_data['id']

# 날짜 데이터 형식 변환
train_data['date'] = train_data['date'].apply(lambda x : str(x[:6])).astype(int)
test_data['date'] = test_data['date'].apply(lambda x : str(x[:6])).astype(int)

# 자연로그 정규화
train_data['price'] = boxcox1p(train_data['price'], 0.01)

# 훈련 데이터에서 타겟(가격) 분리
y = train_data['price']
del train_data['price']

# 로버스트 스케일링
rbst_scaler=RobustScaler()
train_rbst=rbst_scaler.fit_transform(train_data)
test_rbst=rbst_scaler.transform(test_data)

In [8]:
# 최적의 하이퍼파라미터를 위한 그리드 탐색 함수
def grid_search(model, train, y, param_grid, verbose = 2, n_jobs = 5):
    # 그리드 모델 초기화
    grid_model = GridSearchCV(model, param_grid = param_grid, scoring = 'neg_mean_squared_error',
                              cv = 5, verbose = verbose, n_jobs = n_jobs)
    
    # 그리드 모델 학습
    grid_model.fit(train, y)
    
    # 결과 저장
    params = grid_model.cv_results_['params']
    score = grid_model.cv_results_['mean_test_score']
    
    # 데이터 프레임
    results = pd.DataFrame(params)
    results['score'] = score
    
    # RMSLE (Root Mean Squared Log Error) 생성 및 정렬
    results['RMSLE'] = np.sqrt(-1 * results['score'])
    results = results.sort_values('RMSLE')
    
    return results

In [ ]:
# XGB Regressor
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [5, 7, 9, 11, 13, 15],
    'n_estimators': [100, 1000, 5000],
}

xgboost = xgb.XGBRegressor(random_state=1)

grid_search(xgboost, train_data, y, param_grid)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


/opt/conda/lib/python3.9/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warni

In [10]:
# LGBM Regressor
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [5, 13, 21, 29, 37, 45],
    'n_estimators': [10, 50, 100, 300, 500, 700, 900, 1100, 1300, 1500],
    'num_leaves': [8, 16, 32, 64, 128]
}

lightgbm = lgb.LGBMRegressor(random_state=1)

grid_search(lightgbm, train_data, y, param_grid)

Fitting 5 folds for each of 900 candidates, totalling 4500 fits
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=8; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=16; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=16; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=32; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=64; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=128; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=128; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=16; total time=   0.3s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=32; total time=   0.3s
[CV] E

[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=16; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=32; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=64; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=128; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=16; total time=   0.3s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=32; total time=   0.3s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=32; total time=   0.6s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=128; total time=   0.5s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=100, num_leaves

[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=8; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=16; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=32; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=10, num_leaves=64; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=8; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=16; total time=   0.5s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=64; total time=   0.3s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=64; total time=   0.3s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=50, num_leaves=128; total time=   0.3s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=100, num_leaves=8; total time=   0.3s
[CV] END learning_rate=0.01, max_depth=5, n_estimators=100, num_leaves=

[CV] END learning_rate=0.01, max_depth=13, n_estimators=1100, num_leaves=8; total time=   3.7s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1100, num_leaves=16; total time=   4.7s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1100, num_leaves=32; total time=   9.9s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1100, num_leaves=64; total time=  10.9s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1100, num_leaves=128; total time=  15.2s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1300, num_leaves=8; total time=   4.6s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1300, num_leaves=16; total time=   6.7s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1300, num_leaves=32; total time=   6.8s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1300, num_leaves=64; total time=  11.8s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1300, num_leaves=128; total time=  24.8s
[CV] END learning_rate=0.01, max_depth=1

[CV] END learning_rate=0.01, max_depth=13, n_estimators=700, num_leaves=128; total time=  10.0s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=900, num_leaves=8; total time=   4.1s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=900, num_leaves=16; total time=   4.2s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=900, num_leaves=32; total time=   5.1s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=900, num_leaves=64; total time=   9.4s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=900, num_leaves=128; total time=  15.9s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1100, num_leaves=8; total time=   2.9s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1100, num_leaves=16; total time=   7.8s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1100, num_leaves=32; total time=   6.5s
[CV] END learning_rate=0.01, max_depth=13, n_estimators=1100, num_leaves=64; total time=   9.5s
[CV] END learning_rate=0.01, max_depth=13, n_e

...

[CV] END learning_rate=0.1, max_depth=21, n_estimators=500, num_leaves=8; total time=   1.2s
[CV] END learning_rate=0.1, max_depth=21, n_estimators=500, num_leaves=8; total time=   1.1s
[CV] END learning_rate=0.1, max_depth=21, n_estimators=500, num_leaves=16; total time=   1.5s
[CV] END learning_rate=0.1, max_depth=21, n_estimators=500, num_leaves=32; total time=   2.3s
[CV] END learning_rate=0.1, max_depth=21, n_estimators=500, num_leaves=64; total time=   3.6s
[CV] END learning_rate=0.1, max_depth=21, n_estimators=500, num_leaves=64; total time=   4.7s
[CV] END learning_rate=0.1, max_depth=21, n_estimators=500, num_leaves=128; total time=   7.8s
[CV] END learning_rate=0.1, max_depth=21, n_estimators=700, num_leaves=32; total time=   4.0s
[CV] END learning_rate=0.1, max_depth=21, n_estimators=700, num_leaves=64; total time=   6.3s
[CV] END learning_rate=0.1, max_depth=21, n_estimators=700, num_leaves=128; total time=   7.8s
[CV] END learning_rate=0.1, max_depth=21, n_estimators=900, 

[CV] END learning_rate=0.1, max_depth=29, n_estimators=1100, num_leaves=8; total time=   3.5s
[CV] END learning_rate=0.1, max_depth=29, n_estimators=1100, num_leaves=16; total time=   3.7s
[CV] END learning_rate=0.1, max_depth=29, n_estimators=1100, num_leaves=16; total time=   3.5s
[CV] END learning_rate=0.1, max_depth=29, n_estimators=1100, num_leaves=32; total time=   4.7s
[CV] END learning_rate=0.1, max_depth=29, n_estimators=1100, num_leaves=64; total time=   7.8s
[CV] END learning_rate=0.1, max_depth=29, n_estimators=1100, num_leaves=128; total time=  12.4s
[CV] END learning_rate=0.1, max_depth=29, n_estimators=1300, num_leaves=8; total time=   3.0s
[CV] END learning_rate=0.1, max_depth=29, n_estimators=1300, num_leaves=8; total time=   3.0s
[CV] END learning_rate=0.1, max_depth=29, n_estimators=1300, num_leaves=16; total time=   4.3s
[CV] END learning_rate=0.1, max_depth=29, n_estimators=1300, num_leaves=16; total time=   4.1s
[CV] END learning_rate=0.1, max_depth=29, n_estimato

,learning_rate,max_depth,n_estimators,num_leaves,score,RMSLE
596,0.05,45,1500,16,-0.033500,0.183031
446,0.05,21,1500,16,-0.033500,0.183031
546,0.05,37,1500,16,-0.033500,0.183031
496,0.05,29,1500,16,-0.033500,0.183031
591,0.05,45,1300,16,-0.033508,0.183052
...,...,...,...,...,...,...
50,0.01,13,10,8,-0.317956,0.563876
200,0.01,37,10,8,-0.317956,0.563876
150,0.01,29,10,8,-0.317956,0.563876
100,0.01,21,10,8,-0.317956,0.563876


[CV] END learning_rate=0.1, max_depth=45, n_estimators=500, num_leaves=64; total time=   4.3s
[CV] END learning_rate=0.1, max_depth=45, n_estimators=500, num_leaves=128; total time=   8.1s
[CV] END learning_rate=0.1, max_depth=45, n_estimators=700, num_leaves=16; total time=   3.0s
[CV] END learning_rate=0.1, max_depth=45, n_estimators=700, num_leaves=32; total time=   3.1s
[CV] END learning_rate=0.1, max_depth=45, n_estimators=700, num_leaves=64; total time=   6.7s
[CV] END learning_rate=0.1, max_depth=45, n_estimators=700, num_leaves=128; total time=  11.0s
[CV] END learning_rate=0.1, max_depth=45, n_estimators=900, num_leaves=8; total time=   2.2s
[CV] END learning_rate=0.1, max_depth=45, n_estimators=900, num_leaves=16; total time=   2.6s
[CV] END learning_rate=0.1, max_depth=45, n_estimators=900, num_leaves=32; total time=   5.8s
[CV] END learning_rate=0.1, max_depth=45, n_estimators=900, num_leaves=64; total time=   6.5s
[CV] END learning_rate=0.1, max_depth=45, n_estimators=900,

[CV] END learning_rate=0.1, max_depth=37, n_estimators=1100, num_leaves=8; total time=   3.9s
[CV] END learning_rate=0.1, max_depth=37, n_estimators=1100, num_leaves=16; total time=   4.2s
[CV] END learning_rate=0.1, max_depth=37, n_estimators=1100, num_leaves=32; total time=   4.7s
[CV] END learning_rate=0.1, max_depth=37, n_estimators=1100, num_leaves=64; total time=  13.0s
[CV] END learning_rate=0.1, max_depth=37, n_estimators=1100, num_leaves=128; total time=  15.1s
[CV] END learning_rate=0.1, max_depth=37, n_estimators=1300, num_leaves=16; total time=   4.0s
[CV] END learning_rate=0.1, max_depth=37, n_estimators=1300, num_leaves=32; total time=   6.8s
[CV] END learning_rate=0.1, max_depth=37, n_estimators=1300, num_leaves=64; total time=  10.7s
[CV] END learning_rate=0.1, max_depth=37, n_estimators=1300, num_leaves=128; total time=  16.1s
[CV] END learning_rate=0.1, max_depth=37, n_estimators=1500, num_leaves=8; total time=   4.5s
[CV] END learning_rate=0.1, max_depth=37, n_estima